This tutorial is based on the material on [Gravitational Wave Open Science Center](https://www.gw-openscience.org/).

## Preparation

### Download the data

    wget -P data https://www.gw-openscience.org/eventapi/html/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_16KHZ_R1-1126257415-4096.hdf5

All the available open data can be found on [Gravitational Wave Open Science Center](https://www.gw-openscience.org/eventapi/html/allevents/). In this tutorial, we take event GW150914 data as example. In the name of this particular data file:
  * "H-H1" means that the data come from the LIGO Hanford Observatory site and the LIGO "H1" datector;
  * "1126257414-4096" means the data starts at GPS time 1126257414 (Mon Sep 14 09:16:37 GMT 2015), duration 4096 seconds;
    * NOTE: GPS time is number of seconds since Jan 6, 1980 GMT. See http://www.oc.nps.edu/oc2902w/gps/timsys.html.

### Load libraries

In [23]:
#----------------------------------------------------------------
# Load libraries
#----------------------------------------------------------------
import numpy as np
import math
from gwpy.timeseries import TimeSeries

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import h5py

import os

### Set input file and parameters

In [22]:
#----------------------------------------------------------------
# Set parameters
#----------------------------------------------------------------
fn = 'data/H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5' # data file
tevent = 1126259462.422 # Mon Sep 14 09:50:45 GMT 2015
evtname = 'GW150914' # event name

detector = 'H1' # detecotr: L1 or H1

### Load data

In [21]:
#----------------------------------------------------------------
# Load LIGO data
#----------------------------------------------------------------
strain = TimeSeries.read(fn, format='hdf5.losc')
center = int(tevent)
strain = strain.crop(center-16, center+16)

## Analysis on a known event

### First look at data: raw time-series data

In [20]:
#----------------------------------------------------------------
# Show LIGO strain vs. time
#----------------------------------------------------------------
plt.figure()
strain.plot()
plt.ylabel('strain')
plt.show()

The data are dominated by low frequency noise, and there is no way to see a signal here.

### Data in the Fourier domain: ASDs

Plotting these data in the Fourier domain gives us an idea of the frequency content of the data. A way to visualize the frequency content of the data is to plot the amplitude spectral density, ASD. The ASDs are the square root of the power spectral densities (PSDs), which are averages of the square of the fast fourier transforms (FFTs) of the data. They are an estimate of the "strain-equivalent noise" of the detectors versus frequency, which limit the ability of the detectors to identify GW signals.

In [24]:
#----------------------------------------------------------------
# Obtain the power spectrum density PSD / ASD
#----------------------------------------------------------------

asd = strain.asd(fftlength=8)

plt.figure()
asd.plot()
plt.xlim(10, 2000)
plt.ylim(1e-24, 1e-19)
plt.ylabel('ASD (strain/Hz$^{1/2})$')
plt.xlabel('Frequency (Hz)')
plt.show()

You can see strong spectral lines in the data. They are all of instrumental origin. Some are engineered into the detectors (mirror suspension resonances at ~500 Hz and harmonics, calibration lines, control dither lines, etc) and some (60 Hz and harmonics) are unwanted. We'll return to these, later.

You can't see the signal in this plot, since it is relatively weak and less than a second long, while this plot averages over 32 seconds of data. So this plot is entirely dominated by instrumental noise.

### Whitening data
From the ASD above, we can see that noise fluctuations are much larger at low and high frequencies and near spectral lines. We can "whiten" the data, suppressing the extra noise at low frequencies and at the spectral lines, to better see the weak signals in the most sensitive band.

In [25]:
#----------------------------------------------------------------
# Whitening data
#----------------------------------------------------------------

white_data = strain.whiten()

plt.figure()
white_data.plot()
plt.ylabel('strain (whitened)')
plt.show()

### Bandpassing filter
To get rid of remaining high frequency noise, we will also bandpass the data. The macro below shows you a framework to do bandpanssing, but <span style="color:blue">you need to find proper frequency windows by yourself</span>.

In [26]:
#----------------------------------------------------------------
# Bandpass filtering
#----------------------------------------------------------------

white_data_bp = white_data.bandpass(30, 400)

plt.figure()
white_data_bp.plot()
plt.ylabel('strain (whitened + band-pass)')
plt.show()

### Plot a q-transform of the data
Now let's look back to our data. To compare to the analytic model built above, we present our data with a time-frequency spectrogram.

In [29]:
#----------------------------------------------------------------
# q-transform
#----------------------------------------------------------------

dt = 1  #-- Set width of q-transform plot, in seconds
hq = strain.q_transform(outseg=(tevent-dt, tevent+dt))

plt.figure()
fig = hq.plot()
ax = fig.gca()
fig.colorbar(label="Normalised energy")
ax.grid(False)
ax.set_yscale('log')
plt.ylabel('Frequency (Hz)')
plt.show()

### Analytic model (under construction)
Combining knowledge of a little bit of GR and 8.01 physics, we can get the radiation power from this system
\begin{eqnarray}
\frac{dE_{tot}}{dt} & = & -\frac{2}{3}\left(\frac{G^2 M^5_{c}}{32\omega}\right)^{\frac{1}{3}}\dot{\omega}\\
\end{eqnarray}
and the oscilliation frequency as a function of time $t$
\begin{eqnarray}
\omega(t) & = & 2\pi\left(134\rm{Hz}\right)\left(\frac{1.21 M_{\odot}}{M_{c}}\right)^{\frac{5}{8}}\left(\frac{1\rm{s}}{\Delta t}\right)^{\frac{3}{8}}
\end{eqnarray}
Let's take a look at how it looks like.

In [30]:
#----------------------------------------------------------------
# Frequency analytic
#----------------------------------------------------------------

def gwfreq(iM,iT,iT0):
    const = (134.*np.pi*2)*np.power((1.21/iM),5./8.)
    output = const*np.power(np.maximum((iT0-iT),3e-2),-3./8.) # we can max it out above 500 Hz-ish
    return output

times = np.linspace(0., 4., 50)
freq = gwfreq(20, times, 4)

plt.figure()
plt.plot(times, freq)
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()

The yellow line is our signal! Next we look at radiation.
### Analytic model - Wave form
By comibining the radiation power and frequency obtained above, we can get a wave form function
\begin{eqnarray}
f(t) = A(t)\cos(\omega(t)t+\phi)
\end{eqnarray}
For the amplitude variation, we can use the power output. <span style="color:blue">Try to come up with a proper $A(t)$ using radiation power output</span>. I have one answer in my mind, which leads to the shape below.

In [32]:
#----------------------------------------------------------------
# Wave form analytic
#----------------------------------------------------------------

def osc(x,M,t0,n,phi):
    freq = gwfreq(M,x,t0)
    val = n*(np.cos(freq*(t0-x)+phi))*1e-12
    val = val*np.power(M*freq,10./3.)*(1*(x<=t0)+np.exp((freq/(2*np.pi))*(t0-x))*(x>t0))
    return val

def osc_dif(params, x, data, eps):
    iM=params["M"]
    iT0=params["t0"]
    norm=params["n"]
    phi=params["phi"]
    val=osc(x, iM, iT0, norm, phi)
    return (val-data)/eps

times = np.linspace(-0.1, 0.3, 1000)
freq = osc(times, 30, 0.18, 1, 0.0)
plt.figure(figsize=(12, 4))
plt.subplots_adjust(left=0.1, right=0.9, top=0.85, bottom=0.2)
plt.plot(times, freq)
plt.xlabel('Time (s) since '+str(tevent))
plt.ylabel('strain')
plt.show()

Now let's see if it's similar with our data. We zoom in the plot 'Advanced LIGO strain data near GW150914 filtered'.

In [35]:
sample_times = white_data_bp.times.value
sample_data = white_data_bp.value
indxt = np.where((sample_times >= (tevent-0.17)) & (sample_times < (tevent+0.13)))
x = sample_times[indxt]
x = x-x[0]
white_data_bp_zoom = sample_data[indxt]

plt.figure(figsize=(12, 4))
plt.subplots_adjust(left=0.1, right=0.9, top=0.85, bottom=0.2)
plt.plot(x, white_data_bp_zoom)
plt.xlabel('Time (s)')
plt.ylabel('strain (whitened + band-pass)')

import lmfit
from lmfit import Model, minimize, fit_report, Parameters

model = lmfit.Model(osc)
p = model.make_params()
p['M'].set(20)     # Mass guess
p['t0'].set(0.18)  # By construction we put the merger in the center
p['n'].set(1)      # normalization guess
p['phi'].set(0)    # Phase guess
unc = np.full(len(white_data_bp_zoom),20)
out = minimize(osc_dif, params=p, args=(x, white_data_bp_zoom, unc))
print(fit_report(out))
plt.plot(x, model.eval(params=out.params,x=x),'r',label='best fit')
plt.show()

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 81
    # data points      = 1228
    # variables        = 4
    chi-square         = 0.72586206
    reduced chi-square = 5.9302e-04
    Akaike info crit   = -9120.38392
    Bayesian info crit = -9099.93135
[[Variables]]
    M:    17.3561729 +/- 0.20106138 (1.16%) (init = 20)
    t0:   0.18039070 +/- 5.8553e-04 (0.32%) (init = 0.18)
    n:   -0.04145174 +/- 0.00201537 (4.86%) (init = 1)
    phi: -2.09189633 +/- 0.37798446 (18.07%) (init = 0)
[[Correlations]] (unreported correlations are < 0.100)
    C(t0, phi) = -0.981
    C(M, phi)  =  0.893
    C(M, t0)   = -0.825
    C(M, n)    =  0.253
    C(n, phi)  =  0.209
    C(t0, n)   = -0.188


We don't see the oscillation shape because the data is not properly filtered. If you apply a good bandpassing, we will obtain the plot below. <span style="color:blue">Then using the wave form function to fit this distribution, you will get an estimation of the black hole mass.</span>

<img src="plots/GW150914_strain_whiten_filter_fit.png" width=420 align="left"/>

To get the actual form you need to do complicated numerical simulations. <span style="color:blue">As a next step for this project, you can load the template databse and try to tune this function to get the parameters (see template notebook). There are also a number of parameters you can tweak here as well. </span>
## Search signal in long time range
When we zoom in the data above, we look at the small range where the signal is. However in reality we don't know where the signal is in advance, so <span style="color:blue">you need to develop your machinery to search for GW events across a long time range</span>. You can fit one short range by one and see the significance. There is one example below

In [41]:
#----------------------------------------------------------------
# Significance vs. time
#----------------------------------------------------------------

def fitrange(data,xx,tcenter,trange):
    findxt = np.where((xx >= tcenter-trange*0.5) & (xx < tcenter+trange*0.5))
    fwhite_data = data[findxt]
    x = xx[findxt]
    x = x-x[0]
    model = lmfit.Model(osc)
    p = model.make_params()
    p['M'].set(30)
    p['t0'].set(trange*0.5)
    p['n'].set(1)
    p['phi'].set(0)
    unc=np.full(len(fwhite_data),20)
    out = minimize(osc_dif, params=p, args=(x, fwhite_data, unc))
    return abs(out.params["n"].value/out.params["n"].stderr),out.redchi

times = np.arange(-14, 14, 0.05)
times += tevent
sigs=[]
chi2=[]
for time in times:
    pSig,pChi2 = fitrange(white_data_bp.value, sample_times, time, 0.4)
    sigs.append(pSig)
    chi2.append(pChi2)

In [43]:
plt.figure(figsize=(12, 4))
plt.subplots_adjust(left=0.1, right=0.9, top=0.85, bottom=0.2)
plt.plot(times, sigs)
plt.xlabel('Time (s)')
plt.ylabel('N/$\sigma_{N}$')
plt.show()

In [44]:
plt.figure(figsize=(12, 4))
plt.subplots_adjust(left=0.1, right=0.9, top=0.85, bottom=0.2)
plt.plot(times, chi2)
plt.xlabel('Time (s)')
plt.ylabel('$\chi^{2}$')
plt.show()

In this figure we can easily find the signal.
# More things to explore
<span style="color:blue">Additionally, you should pull the Livingston and Virgo data and try to see if you can align the different waveforms. Do you see a good coincidence. How do you quantify. Given the alignment what direction are the GWs coming from? Also, you can pull in the official wave forms and try to tune your model. Additionally, it would be nice to know how far the GW occured? How would you compute the distance? What about the other GW events? What parameters do you see? In particular, what does then neutron star merger look like?</span>